<a href="https://colab.research.google.com/github/nilrey/yolo8-cars/blob/main/Copy_of_cars_yolo8_x_bytrack_frame_1_of_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2

In [ ]:
# Загружаем модель YOLOv8
model = YOLO('yolov8x.pt')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
video_path = '/content/drive/MyDrive/Colab Notebooks/cars/input/s-001.mp4'
cap = cv2.VideoCapture(video_path)
output_path = '/content/drive/MyDrive/Colab Notebooks/cars/output/cars-001-s-yolo8-x-bytrack-frame-1-of-3-001.mp4'


In [ ]:

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

allowed_indices = {2, 3, 5, 6, 7, 8}  # Фильтрация классов автомобилей

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))


In [ ]:
print(fps)

25


In [ ]:
frame_skip = 2  # Пропускаем кадры (анализируем 1 из 3)
frame_count = 0  # Счетчик кадров

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    # Обрабатываем только каждый 3-й кадр
    if frame_count % frame_skip != 0:
        out.write(frame)
        continue

    # Используем модель для анализа текущего кадра с отслеживанием
    results = model.track(frame, persist=True, imgsz=640, iou=0.4) # 0.5

    if results[0].boxes.id is not None:
        for i, box in enumerate(results[0].boxes):
            conf = box.conf[0]
            if int(box.cls[0]) in allowed_indices and conf > 0.5: # 0.7
                xyxy = box.xyxy[0]
                conf = box.conf[0]
                class_name = results[0].names[int(box.cls[0])]
                obj_id = int(results[0].boxes.id[i])  # Получаем ID объекта
                label = f'{class_name} {obj_id} ({conf:.2f})'

                # Рисуем bounding box и ID на кадре
                x1, y1, x2, y2 = map(int, xyxy)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Запись обработанного кадра в выходное видео
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Обработанное видео с трекингом сохранено в {output_path}")
